<a href="https://colab.research.google.com/github/NotShubham1112/Daily-ai-concepts/blob/main/Next-Generation%20Coding%20LLM%20System%3A%20Architecture%20Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Claude Design**

Next-Generation Coding LLM System - Complete Implementation
Optimized for Google Colab (T4/A100 GPU)

Installation Instructions:
1. Open Google Colab
2. Runtime > Change runtime type > T4 GPU
3. Run all cells sequentially
4. Wait for model download (~15 minutes first time)


In [38]:

# ============================================================================
# SECTION 1: INSTALLATION & SETUP
# ============================================================================

# Install required packages with proper versions
import sys
print("Installing dependencies...")
!pip install -q -U transformers accelerate
!pip install -q -U bitsandbytes
!pip install -q sentence-transformers faiss-cpu datasets python-Levenshtein

# Verify bitsandbytes installation
try:
    import bitsandbytes as bnb
    print(f"✓ bitsandbytes {bnb.__version__} installed")
except ImportError:
    print("⚠ Installing bitsandbytes from source...")
    !pip install -q git+https://github.com/TimDettmers/bitsandbytes.git

print("✓ All dependencies installed")

import torch
import json
import ast
import re
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, asdict
from collections import OrderedDict
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# Check GPU availability
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


Installing dependencies...
✓ bitsandbytes 0.49.0 installed
✓ All dependencies installed
GPU Available: True
GPU Name: Tesla T4
GPU Memory: 15.83 GB


In [39]:

# ============================================================================
# SECTION 2: DATA STRUCTURES
# ============================================================================

@dataclass
class Intent:
    task_type: str  # generate, debug, refactor, explain, optimize
    language: str
    complexity: str  # simple, medium, complex
    requires_rag: bool
    requires_execution: bool
    estimated_tokens: int

@dataclass
class Plan:
    goal: str
    constraints: List[str]
    steps: List[Dict]
    success_criteria: str

@dataclass
class VerificationResult:
    valid: bool
    errors: List[str]
    warnings: List[str]
    confidence: float

@dataclass
class SystemOutput:
    code: str
    explanation: str
    assumptions: List[str]
    confidence: float
    tokens_used: int
    retrieval_sources: List[str]


In [40]:

# ============================================================================
# SECTION 3: MOCK KNOWLEDGE BASE (For Testing)
# ============================================================================

# In production, this would be scraped from official docs
KNOWLEDGE_BASE = {
    "python_basics": [
        {
            "content": "Python lists support methods: append(item), extend(iterable), insert(index, item), remove(item), pop(index=-1), clear(), index(item), count(item), sort(), reverse()",
            "source": "python_docs_list",
            "verified": True
        },
        {
            "content": "Python dictionaries support methods: keys(), values(), items(), get(key, default), pop(key), update(dict), clear(). Dict comprehension: {k: v for k, v in items}",
            "source": "python_docs_dict",
            "verified": True
        },
        {
            "content": "File handling: open(file, mode) where mode is 'r' (read), 'w' (write), 'a' (append), 'rb' (read binary). Always use 'with open() as f:' for automatic closing.",
            "source": "python_docs_io",
            "verified": True
        }
    ],
    "pandas": [
        {
            "content": "pandas.DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, suffixes=('_x', '_y')). Parameter 'how' can be: 'inner', 'outer', 'left', 'right', 'cross'",
            "source": "pandas_docs_merge",
            "verified": True
        },
        {
            "content": "pandas.read_csv(filepath, sep=',', header='infer', names=None, index_col=None, usecols=None, dtype=None, na_values=None). Returns DataFrame object.",
            "source": "pandas_docs_read_csv",
            "verified": True
        }
    ],
    "algorithms": [
        {
            "content": "Binary search implementation: def binary_search(arr, target): left, right = 0, len(arr)-1; while left <= right: mid = (left+right)//2; if arr[mid] == target: return mid; elif arr[mid] < target: left = mid+1; else: right = mid-1; return -1",
            "source": "algorithms_binary_search",
            "verified": True
        }
    ]
}

# Flatten knowledge base for embedding
ALL_DOCS = []
for category, docs in KNOWLEDGE_BASE.items():
    for doc in docs:
        doc['category'] = category
        ALL_DOCS.append(doc)

print(f"Loaded {len(ALL_DOCS)} documents into knowledge base")


Loaded 6 documents into knowledge base


In [42]:

# ============================================================================
# SECTION 4: INTENT ROUTER
# ============================================================================

class IntentRouter:
    """Lightweight classifier for task type detection"""

    def __init__(self):
        self.keywords = {
            "generate": ["write", "create", "generate", "implement", "build", "make"],
            "debug": ["fix", "debug", "error", "bug", "wrong", "not working"],
            "refactor": ["refactor", "improve", "optimize", "clean", "rewrite"],
            "explain": ["explain", "what does", "how does", "understand", "clarify"],
            "optimize": ["optimize", "faster", "performance", "speed up", "efficient"]
        }

    def route(self, query: str) -> Intent:
        query_lower = query.lower()

        # Detect task type
        task_type = "generate"  # default
        for task, keywords in self.keywords.items():
            if any(kw in query_lower for kw in keywords):
                task_type = task
                break

        # Detect language
        language = "python"  # default for this demo
        if "javascript" in query_lower or "js" in query_lower:
            language = "javascript"

        # Estimate complexity
        complexity = "simple"
        if len(query.split()) > 20 or "class" in query_lower or "multiple" in query_lower:
            complexity = "medium"
        if "system" in query_lower or "design" in query_lower:
            complexity = "complex"

        # Decide if RAG needed
        requires_rag = task_type in ["generate", "debug", "optimize"]

        # Decide if execution needed
        requires_execution = task_type in ["debug", "optimize"]

        return Intent(
            task_type=task_type,
            language=language,
            complexity=complexity,
            requires_rag=requires_rag,
            requires_execution=requires_execution,
            estimated_tokens=100 if complexity == "simple" else 200
        )


In [43]:

# ============================================================================
# SECTION 5: RAG MODULE
# ============================================================================

class RAGModule:
    """Retrieval-Augmented Generation with semantic search"""

    def __init__(self, docs: List[Dict]):
        print("Initializing RAG module...")
        self.docs = docs

        # Load embedding model (lightweight, CPU-friendly)
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        print("Embedding model loaded")

        # Create embeddings
        self.doc_texts = [doc['content'] for doc in docs]
        print(f"Embedding {len(self.doc_texts)} documents...")
        embeddings = self.embedder.encode(self.doc_texts, show_progress_bar=True)

        # Convert to numpy array with correct dtype
        self.doc_embeddings = np.array(embeddings, dtype=np.float32)

        # Build FAISS index
        dimension = self.doc_embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(self.doc_embeddings)
        print(f"FAISS index built with {self.index.ntotal} vectors")

    def retrieve(self, query: str, top_k: int = 3) -> List[Dict]:
        """Retrieve most relevant documents"""
        # Embed query
        query_embedding = self.embedder.encode([query])
        query_embedding = np.array(query_embedding, dtype=np.float32)

        # Search
        distances, indices = self.index.search(query_embedding, top_k)

        # Return results
        results = []
        for idx, dist in zip(indices[0], distances[0]):
            if idx < len(self.docs):
                doc = self.docs[idx].copy()
                doc['relevance_score'] = float(1 / (1 + dist))  # Convert distance to similarity
                results.append(doc)

        return results

    def format_context(self, results: List[Dict]) -> str:
        """Format retrieved docs as context"""
        if not results:
            return "No relevant documentation found."

        context = "RETRIEVED DOCUMENTATION:\n\n"
        for i, doc in enumerate(results, 1):
            context += f"[{i}] Source: {doc['source']}\n"
            context += f"{doc['content']}\n\n"

        return context


In [44]:

# ============================================================================
# SECTION 6: PLANNER MODULE
# ============================================================================

class PlannerModule:
    """Generates structured execution plans"""

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def generate_plan(self, query: str, intent: Intent, context: str = "") -> Plan:
        """Generate a structured plan for code generation"""

        prompt = f"""You are a code planning assistant. Generate a concise execution plan.

TASK: {query}
LANGUAGE: {intent.language}
TASK TYPE: {intent.task_type}

{context}

Output ONLY a JSON plan with this exact structure:
{{
  "goal": "<one sentence description>",
  "constraints": ["<constraint1>", "<constraint2>"],
  "steps": [
    {{"step": 1, "action": "describe what to do"}},
    {{"step": 2, "action": "describe next action"}}
  ],
  "success_criteria": "<how to verify success>"
}}

Plan:"""

        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=0.3,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        plan_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract JSON from response
        try:
            # Find JSON in response
            json_match = re.search(r'\{.*\}', plan_text, re.DOTALL)
            if json_match:
                plan_dict = json.loads(json_match.group())
                return Plan(**plan_dict)
        except:
            pass

        # Fallback plan
        return Plan(
            goal=query,
            constraints=["Use only standard library" if "pandas" not in query.lower() else "Use pandas"],
            steps=[
                {"step": 1, "action": "understand requirements"},
                {"step": 2, "action": "generate code"},
                {"step": 3, "action": "verify syntax"}
            ],
            success_criteria="Code runs without errors"
        )


In [45]:

# ============================================================================
# SECTION 7: VERIFIER MODULE
# ============================================================================

class VerifierModule:
    """Multi-stage code verification"""

    @staticmethod
    def verify_syntax(code: str) -> Tuple[bool, Optional[str]]:
        """Check Python syntax"""
        try:
            ast.parse(code)
            return True, None
        except SyntaxError as e:
            return False, f"Syntax error at line {e.lineno}: {e.msg}"

    @staticmethod
    def verify_apis(code: str, retrieved_docs: List[Dict]) -> Tuple[bool, List[str]]:
        """Check if APIs used exist in documentation"""
        # Extract function calls from code
        try:
            tree = ast.parse(code)
            calls = set()

            for node in ast.walk(tree):
                if isinstance(node, ast.Call):
                    if isinstance(node.func, ast.Name):
                        calls.add(node.func.id)
                    elif isinstance(node.func, ast.Attribute):
                        calls.add(node.func.attr)

            # Build list of known APIs from retrieved docs
            known_apis = set()
            for doc in retrieved_docs:
                # Extract function names from doc content
                doc_functions = re.findall(r'(\w+)\(', doc['content'])
                known_apis.update(doc_functions)

            # Standard library functions (whitelist)
            stdlib = {'print', 'len', 'range', 'enumerate', 'zip', 'map', 'filter',
                     'open', 'sorted', 'sum', 'min', 'max', 'abs', 'round', 'int',
                     'str', 'list', 'dict', 'set', 'tuple', 'type', 'isinstance'}
            known_apis.update(stdlib)

            # Check for unknown APIs
            unknown = [api for api in calls if api not in known_apis and not api.startswith('_')]

            return len(unknown) == 0, unknown
        except:
            return True, []  # If parsing fails, don't block

    @staticmethod
    def verify(code: str, retrieved_docs: List[Dict]) -> VerificationResult:
        """Complete verification pipeline"""
        errors = []
        warnings = []

        # Stage 1: Syntax
        syntax_valid, syntax_error = VerifierModule.verify_syntax(code)
        if not syntax_valid:
            errors.append(syntax_error)
            return VerificationResult(valid=False, errors=errors, warnings=warnings, confidence=0.0)

        # Stage 2: API validation
        apis_valid, unknown_apis = VerifierModule.verify_apis(code, retrieved_docs)
        if not apis_valid:
            warnings.append(f"Unknown APIs detected: {', '.join(unknown_apis)}")

        # Calculate confidence
        confidence = 0.9 if apis_valid else 0.6

        return VerificationResult(
            valid=syntax_valid,
            errors=errors,
            warnings=warnings,
            confidence=confidence
        )


In [46]:

# ============================================================================
# SECTION 8: EXECUTION ENGINE
# ============================================================================

class ExecutionEngine:
    """Code generation with LLM"""

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def generate(self, query: str, plan: Plan, context: str) -> str:
        """Generate code based on plan and context"""

        prompt = f"""You are an expert Python programmer. Generate clean, correct code.

TASK: {query}

PLAN:
Goal: {plan.goal}
Steps: {', '.join([s['action'] for s in plan.steps])}

{context}

CRITICAL RULES:
1. Use ONLY functions and methods mentioned in the documentation above
2. Do NOT invent or guess API names
3. Write complete, runnable code
4. Include necessary imports
5. Add brief comments for clarity

Generate the complete code now:

```python"""

        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=300,
                temperature=0.2,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )

        generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract code from response
        code = self._extract_code(generated)

        return code

    def _extract_code(self, text: str) -> str:
        """Extract Python code from model output"""
        # Look for code block
        code_match = re.search(r'```python\n(.*?)```', text, re.DOTALL)
        if code_match:
            return code_match.group(1).strip()

        # Look for code after "```python"
        if '```python' in text:
            code = text.split('```python')[-1].split('```')[0].strip()
            return code

        # Try to extract code after the prompt
        lines = text.split('\n')
        code_lines = []
        in_code = False

        for line in lines:
            if line.strip().startswith(('import ', 'from ', 'def ', 'class ', '#')):
                in_code = True
            if in_code:
                code_lines.append(line)

        if code_lines:
            return '\n'.join(code_lines)

        return text.strip()


In [47]:

# ============================================================================
# SECTION 9: MAIN SYSTEM
# ============================================================================

class CodingLLMSystem:
    """Complete coding intelligence system"""

    def __init__(self):
        print("\n" + "="*70)
        print("INITIALIZING CODING LLM SYSTEM")
        print("="*70)

        # Load model
        print("\n[1/5] Loading LLM model...")
        from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

        model_name = "Qwen/Qwen2.5-Coder-1.5B-Instruct"  # Lighter model for Colab

        # Try with quantization first, fallback to FP16 if bitsandbytes fails
        try:
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4"
            )

            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=quantization_config,
                device_map="auto",
                trust_remote_code=True
            )
            print("✓ Model loaded successfully (4-bit quantized)")

        except Exception as e:
            print(f"⚠ Quantization failed: {e}")
            print("  Loading model in FP16 instead...")

            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True
            )
            print("✓ Model loaded successfully (FP16)")

        # Initialize modules
        print("\n[2/5] Initializing Intent Router...")
        self.router = IntentRouter()
        print("✓ Router ready")

        print("\n[3/5] Initializing RAG Module...")
        self.rag = RAGModule(ALL_DOCS)
        print("✓ RAG ready")

        print("\n[4/5] Initializing Planner...")
        self.planner = PlannerModule(self.model, self.tokenizer)
        print("✓ Planner ready")

        print("\n[5/5] Initializing Execution Engine...")
        self.executor = ExecutionEngine(self.model, self.tokenizer)
        print("✓ Executor ready")

        self.verifier = VerifierModule()

        print("\n" + "="*70)
        print("SYSTEM READY!")
        print("="*70 + "\n")

    def process(self, query: str, verbose: bool = True) -> SystemOutput:
        """Main processing pipeline"""

        if verbose:
            print(f"\n{'='*70}")
            print(f"QUERY: {query}")
            print(f"{'='*70}\n")

        # Step 1: Route
        if verbose:
            print("[STEP 1] Routing query...")
        intent = self.router.route(query)
        if verbose:
            print(f"  → Task: {intent.task_type}, Language: {intent.language}, Complexity: {intent.complexity}")

        # Step 2: Retrieve
        retrieved_docs = []
        context = ""
        if intent.requires_rag:
            if verbose:
                print("\n[STEP 2] Retrieving documentation...")
            retrieved_docs = self.rag.retrieve(query, top_k=3)
            context = self.rag.format_context(retrieved_docs)
            if verbose:
                print(f"  → Retrieved {len(retrieved_docs)} documents")
                for doc in retrieved_docs:
                    print(f"     - {doc['source']} (relevance: {doc['relevance_score']:.2f})")

        # Step 3: Plan
        if verbose:
            print("\n[STEP 3] Generating execution plan...")
        plan = self.planner.generate_plan(query, intent, context)
        if verbose:
            print(f"  → Goal: {plan.goal}")
            print(f"  → Steps: {len(plan.steps)}")

        # Step 4: Generate
        if verbose:
            print("\n[STEP 4] Generating code...")
        code = self.executor.generate(query, plan, context)
        if verbose:
            print(f"  → Generated {len(code.split())} tokens")

        # Step 5: Verify
        if verbose:
            print("\n[STEP 5] Verifying code...")
        verification = self.verifier.verify(code, retrieved_docs)
        if verbose:
            print(f"  → Valid: {verification.valid}")
            print(f"  → Confidence: {verification.confidence:.2f}")
            if verification.errors:
                print(f"  → Errors: {verification.errors}")
            if verification.warnings:
                print(f"  → Warnings: {verification.warnings}")

        # Synthesize output
        output = SystemOutput(
            code=code,
            explanation=plan.goal,
            assumptions=plan.constraints,
            confidence=verification.confidence,
            tokens_used=intent.estimated_tokens,
            retrieval_sources=[doc['source'] for doc in retrieved_docs]
        )

        return output


In [48]:
# ============================================================================
# SECTION 10: INITIALIZE SYSTEM
# ============================================================================

# Create system instance
system = CodingLLMSystem()



INITIALIZING CODING LLM SYSTEM

[1/5] Loading LLM model...
✓ Model loaded successfully (4-bit quantized)

[2/5] Initializing Intent Router...
✓ Router ready

[3/5] Initializing RAG Module...
Initializing RAG module...
Embedding model loaded
Embedding 6 documents...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

FAISS index built with 6 vectors
✓ RAG ready

[4/5] Initializing Planner...
✓ Planner ready

[5/5] Initializing Execution Engine...
✓ Executor ready

SYSTEM READY!



In [49]:
# ============================================================================
# SECTION 11: TEST CASES
# ============================================================================

def run_tests():
    """Run example test cases"""

    test_queries = [
        "Write a function to merge two dictionaries in Python",
        "Create a function to read a CSV file and return a pandas DataFrame",
        "Implement binary search algorithm",
        "Write code to find duplicates in a list",
        "Create a function that reverses a string"
    ]

    print("\n" + "="*70)
    print("RUNNING TEST CASES")
    print("="*70)

    results = []

    for i, query in enumerate(test_queries, 1):
        print(f"\n\n{'#'*70}")
        print(f"TEST CASE {i}/{len(test_queries)}")
        print(f"{'#'*70}")

        try:
            output = system.process(query, verbose=True)

            print(f"\n{'='*70}")
            print("FINAL OUTPUT")
            print(f"{'='*70}")
            print("\nCODE:")
            print("-" * 70)
            print(output.code)
            print("-" * 70)
            print(f"\nConfidence: {output.confidence:.2f}")
            print(f"Sources: {', '.join(output.retrieval_sources)}")

            results.append({
                "query": query,
                "success": True,
                "confidence": output.confidence,
                "code_length": len(output.code)
            })

        except Exception as e:
            print(f"\n❌ ERROR: {str(e)}")
            results.append({
                "query": query,
                "success": False,
                "error": str(e)
            })

    # Summary
    print("\n\n" + "="*70)
    print("TEST SUMMARY")
    print("="*70)

    successful = sum(1 for r in results if r.get('success'))
    print(f"\nTotal Tests: {len(results)}")
    print(f"Successful: {successful}")
    print(f"Failed: {len(results) - successful}")
    print(f"Success Rate: {successful/len(results)*100:.1f}%")

    if successful > 0:
        avg_confidence = np.mean([r['confidence'] for r in results if r.get('success')])
        print(f"Average Confidence: {avg_confidence:.2f}")

In [53]:

# ============================================================================
# SECTION 12: INTERACTIVE MODE
# ============================================================================

def interactive_mode():
    """Interactive query interface"""
    print("\n" + "="*70)
    print("INTERACTIVE MODE")
    print("="*70)
    print("\nEnter your coding queries (type 'exit' to quit)\n")

    while True:
        query = input("\n🤖 Query: ").strip()

        if query.lower() in ['exit', 'quit', 'q']:
            print("\nGoodbye!")
            break

        if not query:
            continue

        try:
            output = system.process(query, verbose=False)

            print("\n" + "="*70)
            print("CODE:")
            print("="*70)
            print(output.code)
            print("\n" + "="*70)
            print(f"Confidence: {output.confidence:.2f}")
            if output.retrieval_sources:
                print(f"Sources: {', '.join(output.retrieval_sources)}")

        except Exception as e:
            print(f"\n❌ Error: {str(e)}")


In [54]:

# ============================================================================
# SECTION 13: MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("\n" + "="*70)
    print("CODING LLM SYSTEM - READY FOR TESTING")
    print("="*70)
    print("\nChoose an option:")
    print("1. Run automated tests")
    print("2. Interactive mode")
    print("3. Single query demo")

    # For Colab, we'll run a single demo automatically
    print("\n[Running single query demo...]\n")

    demo_query = "Write a function to find the maximum value in a list"
    output = system.process(demo_query, verbose=True)

    print(f"\n{'='*70}")
    print("GENERATED CODE")
    print(f"{'='*70}")
    print(output.code)
    print(f"\n{'='*70}")
    print(f"Confidence: {output.confidence:.2f}")
    print(f"{'='*70}")

    print("\n✅ System is working! You can now:")
    print("   - Call: run_tests() to run all test cases")
    print("   - Call: interactive_mode() for interactive queries")
    print("   - Call: system.process('your query') for single queries")


CODING LLM SYSTEM - READY FOR TESTING

Choose an option:
1. Run automated tests
2. Interactive mode
3. Single query demo

[Running single query demo...]


QUERY: Write a function to find the maximum value in a list

[STEP 1] Routing query...
  → Task: generate, Language: python, Complexity: simple

[STEP 2] Retrieving documentation...
  → Retrieved 3 documents
     - python_docs_list (relevance: 0.46)
     - algorithms_binary_search (relevance: 0.38)
     - python_docs_dict (relevance: 0.37)

[STEP 3] Generating execution plan...
  → Goal: Write a function to find the maximum value in a list
  → Steps: 3

[STEP 4] Generating code...
  → Generated 113 tokens

[STEP 5] Verifying code...
  → Valid: True
  → Confidence: 0.60
  → Warnings: ['Unknown APIs detected: find_maximum_value']

GENERATED CODE
# Import necessary libraries
from typing import List

def find_maximum_value(numbers: List[int]) -> int:
    """
    Find the maximum value in a given list of integers.
    
    Args:
    numb

In [51]:
run_tests()


RUNNING TEST CASES


######################################################################
TEST CASE 1/5
######################################################################

QUERY: Write a function to merge two dictionaries in Python

[STEP 1] Routing query...
  → Task: generate, Language: python, Complexity: simple

[STEP 2] Retrieving documentation...
  → Retrieved 3 documents
     - python_docs_dict (relevance: 0.56)
     - pandas_docs_merge (relevance: 0.47)
     - python_docs_list (relevance: 0.43)

[STEP 3] Generating execution plan...
  → Goal: Write a function to merge two dictionaries in Python
  → Steps: 3

[STEP 4] Generating code...
  → Generated 95 tokens

[STEP 5] Verifying code...
  → Valid: True
  → Confidence: 0.60
  → Warnings: ['Unknown APIs detected: merge_dictionaries']

FINAL OUTPUT

CODE:
----------------------------------------------------------------------
# Import necessary libraries
import pandas as pd

def merge_dictionaries(dict1, dict2):
    """
    M

In [55]:
# Simplest way - just run this
system.process('python code for swapping 2 variables')


QUERY: python code for swapping 2 variables

[STEP 1] Routing query...
  → Task: generate, Language: python, Complexity: simple

[STEP 2] Retrieving documentation...
  → Retrieved 3 documents
     - pandas_docs_merge (relevance: 0.45)
     - python_docs_dict (relevance: 0.44)
     - python_docs_list (relevance: 0.44)

[STEP 3] Generating execution plan...
  → Goal: python code for swapping 2 variables
  → Steps: 3

[STEP 4] Generating code...
  → Generated 95 tokens

[STEP 5] Verifying code...
  → Valid: True
  → Confidence: 0.60
  → Warnings: ['Unknown APIs detected: swap_variables']


SystemOutput(code='# Import necessary libraries\nimport pandas as pd\n\n# Define a function to swap two variables\ndef swap_variables(a, b):\n    # Swap the values of a and b using a temporary variable\n    temp = a\n    a = b\n    b = temp\n    return a, b\n\n# Example usage\nif __name__ == "__main__":\n    # Create two sample variables\n    x = 5\n    y = 10\n    \n    # Print original values\n    print("Before swapping:")\n    print(f"x = {x}, y = {y}")\n    \n    # Call the swap_variables function\n    x, y = swap_variables(x, y)\n    \n    # Print swapped values\n    print("\\nAfter swapping:")\n    print(f"x = {x}, y = {y}")', explanation='python code for swapping 2 variables', assumptions=['Use only standard library'], confidence=0.6, tokens_used=100, retrieval_sources=['pandas_docs_merge', 'python_docs_dict', 'python_docs_list'])

In [ ]:
interactive_mode()


INTERACTIVE MODE

Enter your coding queries (type 'exit' to quit)


🤖 Query: You are optimized for algorithmic problem solving. Favor asymptotically efficient solutions, clear complexity analysis, and correctness proofs where appropriate.

CODE:
# Import necessary libraries
import numpy as np

def calculate_average(numbers):
    """
    Calculate the average of a list of numbers using numpy's mean function.
    
    Args:
    numbers (list): A list of numerical values.
    
    Returns:
    float: The average of the numbers.
    """
    # Check if the input is a list
    if not isinstance(numbers, list):
        raise ValueError("Input must be a list.")
    
    # Check if all elements in the list are numeric
    if not all(isinstance(num, (int, float)) for num in numbers):
        raise ValueError("All elements in the list must be numeric.")
    
    # Calculate the average using numpy's mean function
    average = np.mean(numbers)
    
    return average

# Example usage
numbers = [

KeyboardInterrupt: Interrupted by user